In [1]:

from pprint import pprint

file = 'cmip.json'
base = 'https://wcrp-cmip.github.io/WCRP-universe/activity/'    

fullpath = base + file

----
### Using the CMIPLD library

Pros: 
- Easier to use, and has additional tools to aid clarity and processing. 

Cons: 
- Currently under development, so will need to be regularly upgraded until a stable version is released. 


In [2]:
from cmipld import processor

#  here we can use the shorthand definitions of the repositories (their prefixes). of the fullpath as is easier for the end user. 

processor.get( f'universal:activity/{file}')

ImportError: cannot import name 'processor' from 'cmipld' (/Users/daniel.ellis/WIPwork/CMIP-LD/cmipld/__init__.py)

----
### Getting a file with PyLD
Pros:
- The approved Python library for handling JSONLD

Cons: 
- May Lack some of the features the CMIPLD wrapper has incorporated.  

In [3]:
from pyld import jsonld

In [4]:
# for a simplified (compacted view) of the json-ld file

'''
usually we would explicitly define the context for a file, which is located in the same directory as the file
compact = jsonld.compact(f'{base}{file}',f'{base}_context_')
however as the context is already defined in the file, we can repeat the file URL instead.
'''


compacted = jsonld.compact(fullpath, fullpath)

# we can also optionally remove the context and the type information
del compacted['@context'],  compacted['type']

pprint(compacted)

{'description': 'CMIP DECK: 1pctCO2, abrupt-4xCO2, amip, esm-piControl, '
                'esm-historical, historical, and piControl experiments',
 'id': 'universal:activity/cmip',
 'name': 'CMIP',
 'url': 'https://gmd.copernicus.org/articles/9/1937/2016/gmd-9-1937-2016.pdf'}


------
### Getting a file with requests 
Pros:
- A simple get request. Does not require any additional librarys or processing and available in all languages. 

Cons: 
- This only fetches the json file, and does not leverage any of the JSONLD tools. 


In [5]:
import requests
contents = requests.get(fullpath).json()
pprint(contents)

{'@context': '_context_',
 'description': 'CMIP DECK: 1pctCO2, abrupt-4xCO2, amip, esm-piControl, '
                'esm-historical, historical, and piControl experiments',
 'id': 'cmip',
 'name': 'CMIP',
 'type': 'activity',
 'url': 'https://gmd.copernicus.org/articles/9/1937/2016/gmd-9-1937-2016.pdf'}


In [6]:
import cmipld 
self = cmipld.processor 
compact: bool = True,
expand_ctx: bool = True,
expand_links: bool = True,
no_ctx: bool = False,
as_json: bool = False,
pprint: bool = False,
is_nested: bool = False

jsonld_doc = fullpath

In [7]:
expanded = jsonld.expand(jsonld_doc, options={'defaultLoader': self.loader})
expanded

[{'https://wcrp-cmip.github.io/WCRP-universe/activity/description': [{'@value': 'CMIP DECK: 1pctCO2, abrupt-4xCO2, amip, esm-piControl, esm-historical, historical, and piControl experiments'}],
  '@id': 'https://wcrp-cmip.github.io/WCRP-universe/activity/cmip',
  'https://wcrp-cmip.github.io/WCRP-universe/activity/name': [{'@value': 'CMIP'}],
  '@type': ['https://wcrp-cmip.github.io/WCRP-universe/activity/activity'],
  'https://wcrp-cmip.github.io/WCRP-universe/activity/url': [{'@value': 'https://gmd.copernicus.org/articles/9/1937/2016/gmd-9-1937-2016.pdf'}]}]

In [8]:

item = expanded[0]
item

{'https://wcrp-cmip.github.io/WCRP-universe/activity/description': [{'@value': 'CMIP DECK: 1pctCO2, abrupt-4xCO2, amip, esm-piControl, esm-historical, historical, and piControl experiments'}],
 '@id': 'https://wcrp-cmip.github.io/WCRP-universe/activity/cmip',
 'https://wcrp-cmip.github.io/WCRP-universe/activity/name': [{'@value': 'CMIP'}],
 '@type': ['https://wcrp-cmip.github.io/WCRP-universe/activity/activity'],
 'https://wcrp-cmip.github.io/WCRP-universe/activity/url': [{'@value': 'https://gmd.copernicus.org/articles/9/1937/2016/gmd-9-1937-2016.pdf'}]}

In [9]:
processed_item = self._resolve_ids(item, compact).copy()
pprint(processed_item)

TypeError: 'tuple' object is not callable

In [ ]:
self.compact(processed_item)

In [ ]:
from pyld import jsonld 
jsonld.compact(fullpath, fullpath)

In [ ]:
from pyld.jsonld import JsonLdProcessor
from urllib.parse import urljoin
import requests

def fetch_resolved_context(jsonld_uri):
    # Fetch the JSON-LD document from the given URI
    document = requests.get(jsonld_uri).json()

    # Extract the @context field
    context = document.get('@context')
    if not context:
        raise ValueError("No @context found in the JSON-LD document.")

    # Resolve the context URI if it is relative
    if isinstance(context, str):
        # Resolve relative URI against the base JSON-LD document URI
        context_uri = urljoin(jsonld_uri, context)
        resolved_context = requests.get(context_uri).json()
    elif isinstance(context, dict):
        # Inline context, return it as is
        resolved_context = context
    else:
        raise ValueError("Complex @context structures (e.g., arrays) are not supported in this implementation.")

    return resolved_context

# Example usage
jsonld_uri = fullpath
try:
    resolved_context = fetch_resolved_context(jsonld_uri)
    print("Resolved Context Content:")
    print(resolved_context)
except Exception as e:
    print(f"Error: {e}")

jsonld.expand(resolved_context['@context'])

In [ ]:
ctx = jsonld.compact(fullpath,fullpath)['@context']



In [ ]:
jsonld.load_document(fullpath,options={"documentLoader": jsonld._default_document_loader})

In [ ]:
from urllib.parse import urljoin
import requests

def resolve_contexts(contexts, base_uri):
    """
    Resolves and expands @context URIs or inline contexts recursively.
    
    :param contexts: A single @context (str, list, or dict).
    :param base_uri: The base URI to resolve relative paths.
    :return: Fully resolved context as a list of dicts.
    """
    if isinstance(contexts, str):
        # Resolve a single string URI
        
        resolved_uri = urljoin(base_uri, contexts)
        print('Resolved',resolved_uri)
        return fetch_resolved_context(resolved_uri)
    
    elif isinstance(contexts, list):
        # Resolve each item in the list
        resolved_list = []
        for ctx in contexts:
            resolved_list.extend(resolve_contexts(ctx, base_uri))
        return resolved_list

    elif isinstance(contexts, dict):
        # Inline context, return as-is
        return [contexts]
    
    else:
        raise ValueError("Unsupported @context format.")

def combine(context):
    if isinstance(context, list):
        newctx = {}
        
        for i in context:
            if isinstance(i, dict):
                newctx.update(i)
            # elif isinstance(i, list):
            #     newctx.update(combine(i))
            # else: 
            #     raise ValueError("Unsupported @context format.",i,context)
        # context = newctx
        return newctx
    elif isinstance(context,dict):
        return context

def fetch_resolved_context(jsonld_uri):
    """
    Fetch a JSON-LD document and resolve its @context field.
    
    :param jsonld_uri: The URI of the JSON-LD document.
    :return: The JSON-LD document with resolved contexts.
    """
    # Fetch the JSON-LD document
    response = requests.get(jsonld_uri)
    response.raise_for_status()
    document = response.json()

    # Extract and resolve @context
    context = document.get('@context')
    if context:
        # Resolve all contexts (whether string, list, or inline)
        resolved_context = resolve_contexts(context, jsonld_uri)
        context = resolved_context  # Update the document with resolved contexts

        # context = combine(context)
        # context = combine(context)
        
        
    return context

def get_context(jsonld_uri):
    """
    Fetch a JSON-LD document and extract its @context field.
    
    :param jsonld_uri: The URI of the JSON-LD document.
    :return: The extracted @context field.
    """
    ctx_list = fetch_resolved_context(jsonld_uri)

    # Combine all contexts into a single dict
    context = combine(ctx_list)
    return context
    
# Example usage
jsonld_uri = fullpath
try:
    resolved_document = get_context(jsonld_uri)
    
    print("Document with Resolved Contexts:")
    pprint(resolved_document)
except Exception as e:
    print(f"Error: {e}")
